## Generate an 3D model of grains distributed in a volume

This script is written to prepare a nanoscopic 3D model of hydrogenated amorphous silicon (a-Si:H). Besides this purpose the script can be used to randomly place grains defind by xyz coordinates in a pre defined volume. 
A volume is defined by the variables box_XY_Length and box_Z_Length. 
The main idea is to read batches of xyz positions from *.cvs files in a secondary directory ("DLA_Cluster_ca100" in this example). Here, these batches represen aggregates from diffusion limited aggregation (DLA) of grains defined by one xyz coordinat. The DLA were aggregated in around a xyz start point of 50,50,50. To move the coordination system of the DLA to the origin of the pre defined volume the variable start ist used. 
The a-Si:H model contains a second type of grains. This second type appears not to be arranged in DLA but to appear in pair, triplets and so on. They are placed randomly as well. 
Export files of the xyz positions of grains type one, type two and of the not occupied volume (called void here) are written. 

import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import os
import glob
import random
import time

Define a volume and distribute baches of xyz (lacated at data_location) coordinates in this volume randomly.

In [6]:
#supposing the xyz batch is in a subdirectory of the project 
#give the name of the directory here.
data_location = "DLA_Cluster_ca100" 
file_structure = "*.csv" #so later seach for the identifier .cvs
outFileName = "DLAmodel_Cl200_XY140_Z20_Particles" # name the output file
outFileTyp = ".csv" # a text file will be saved anyway

headerLen = 1 # number of header lines
path = data_location + os.sep + file_structure
outFile = outFileName + outFileTyp

# Define the xy volume in which the DLA should be distributed
box_XY_Length = 140 # XY Box Size
box_Z_Length = 20 # Z Box Size
limit = 250 # limit must be a higher number than files in the directory

start = -50 # move axis center
stopXY = start + box_XY_Length
stopZ = start + box_Z_Length


t1_start = time.time()

# make random distrubution array
randListX = [random.randint(start,stopXY) for iter in range(limit)]
randListY = [random.randint(start,stopXY) for iter in range(limit)]
randListZ = [random.randint(start,stopZ) for iter in range(limit)]
arr0 = np.zeros((limit,),dtype=int)
arrX = np.array(randListX,dtype=int)
arrY = np.array(randListY,dtype=int)
arrZ = np.array(randListZ,dtype=int)
arrVec = np.stack((arr0,arrX,arrY,arrZ)).transpose()
vector = iter(np.array(arrVec))

t2_stop = time.time()
#print(arrVec)

# grep all files from the folder and distribute them in a volme
merg = np.array([[0,0,0,0],[0,0,0,0]]) # will be deleted later

t3_start = time.time()

for filepath in glob.iglob(path):
    #print(filepath)
    data = np.loadtxt(filepath, delimiter=',', dtype = int, skiprows=headerLen)
    data = data + next(vector)
    merg = np.append(merg,data,axis = 0)
merg = np.delete(merg,[0,1],0)

t4_stop = time.time()

partDF = pd.DataFrame(merg, columns=["Counter","X","Y","Z"])
partDF = partDF.drop(columns=["Counter"])
partDF = partDF.drop_duplicates(keep="first")

print( )
print("Length of generated File", len(merg))
print()
#------Save to file-------
partDF.to_csv(outFile,index=False)
#-------------------------

print("Name of exported file:", outFile)
print()
print("Random distribution array construction = ", t2_stop - t1_start , "seconds")
print("Loop over all files = ", t4_stop - t3_start , "seconds")


Length of generated File 20200

Name of exported file: DLAmodel_Cl200_XY140_Z20_Particles.csv

Random distribution array construction =  0.0009970664978027344 seconds
Loop over all files =  0.21592998504638672 seconds


 Randomly position clusters of the second type of grains (voids).


In [14]:
#find the number of grains of the first type
nDOD = len(partDF)
#the second type of grains appear 0.031 times as much as the first type
nVac = int(round(nDOD*0.031,0))

vacX_list = list()
vacY_list = list()
vacZ_list = list()

nVIter = iter(list(range(nVac)))

#position a number of grains of second type (nVac) in small clusters of 1, 2, or 3 grains
#clSize defines the number of grains in a cluster
for element in nVIter:
    clSize = random.randint(1,3)
    clSizeList = list(range(clSize-1))
    xPos = random.randint(1,box_XY_Length)
    yPos = random.randint(1,box_XY_Length)
    zPos = random.randint(1,box_Z_Length)
    
    vacX_list.append(xPos)
    vacY_list.append(yPos)
    vacZ_list.append(zPos)
    
    #print("Cluster size:", clSize)
    #print(element,"xyz:" , xPos, yPos, zPos)
    for element in clSizeList:
        position = next(nVIter)
        xPos = random.randint(xPos-1,xPos+1)
        yPos = random.randint(yPos-1,yPos+1)
        zPos = random.randint(zPos-1,zPos+1)
        
        vacX_list.append(xPos)
        vacY_list.append(yPos)
        vacZ_list.append(zPos)
        #print(position,"xyz:" , xPos, yPos, zPos)

In [15]:
vacDF = pd.DataFrame({"X": vacX_list,
                      "Y": vacY_list,
                      "Z": vacZ_list})
vacDF = vacDF.drop_duplicates(keep="first")
print("Number of grains type two:", len(vacDF))

outFileName = "DLAmodel_Cl200_XY140_Z20_Vacancies" # name the generated file
outFileTyp = ".csv" # a text file will be saved anyway
outFile = outFileName + outFileTyp
vacDF.to_csv(outFile,index=False)

Number of grains type two: 598


Export a file that contains all xyz positions not occupied by grains of type one or two. 

In [16]:
# Make a file of the not occupied space

listX = list(range(-5,box_XY_Length+5+1))
listY = list(range(-5,box_XY_Length+5+1))
listZ = list(range(-5,box_Z_Length+5+1))
#find all combinations of listsXYZ
volumeArray = np.array(np.meshgrid(listX, listY, listZ)).T.reshape(-1,3)
#make pandas DataFrame from this
volumeDF = pd.DataFrame(volumeArray, columns=["X","Y","Z"])
volumeDF = volumeDF.drop_duplicates(keep="first")

voidDF = pd.concat([volumeDF,partDF,vacDF]).drop_duplicates(keep=False)

outFileName_volume = "DLAmodel_Cl200_XY140_Z20_Volume" # name the generated file
outFileTyp = ".csv" # a text file will be saved anyway
outFile = outFileName_volume + outFileTyp

voidDF.to_csv(outFile,index=False)

In [18]:
print("xyz positions on volume:",len(volumeDF))
print("Number of grains type one:",len(partDF))
print("Number of grains type two:",len(vacDF))
print("Number of not occupied positions:",len(voidDF))

xyz positions on volume: 706831
Number of grains type one: 19687
Number of grains type two: 598
Number of not occupied positions: 686640
